In [ ]:
import pandas as pd
import csv
from datetime import datetime

## Merging daily files into one file and creating counts

This code is used to merge all the files containing the tweets filtered according to a keyword into one file and then creating a count. This code produces a daily as well as an hourly count. For our project we ended up using only the daily count, as it had to match the Binance and Google Trend data.

In [ ]:
# Function to loop through my files and merge them according to the keyword 

def merging_total(keyword):
    # I left my real file names in here, because I think it makes it easier to understand the structure of the function
    # and why I decided to write it as it is
    
    # Reading in the first keyword file and creating a dataframe of it
    df= pd.read_csv('ukraine_russia-2022-02-24_'+keyword+'.csv', header=[0])
    
    file_start= 'ukraine_russia-2022-'
    month_02 = '02-'  # for the February files
    
    # I read in the first file and extend the data from the second file on
    # My files start on the 24th, so i append from the second file on (the 25th)
    i = 25
    
    while i<29:
        file=file_start+month_02+str(i)+'_'+keyword+'.csv'
        df_add=pd.read_csv(file, header=[0])
        df = pd.concat([df, df_add], ignore_index=True)
        i+=1
    
    month_03 = '03-' # for the March files
    m = 1
    
    # Because my files use dates with the days numbered as 01, 02, 03 and so on I used two while loops
    # as the loop goes 1, 2, 3, ..., 9 and not 01, 02, 03, ..., 09
    while m < 10:
        file=file_start+month_03+'0'+str(m)+'_'+keyword+'.csv' # adding the 0 in front of m to match the files
        df_add=pd.read_csv(file, header=[0])
        df = pd.concat([df, df_add], ignore_index=True)
        m+=1
    
    # from the 10. on, there is no need for the added 0 anymore
    while m<25:
        file=file_start+month_03+str(m)+'_'+keyword+'.csv' 
        df_add=pd.read_csv(file, header=[0])
        df = pd.concat([df, df_add], ignore_index=True)
        m+=1
    
    return df   
        
    # this function returns a dataframe with all tweets containing the chosen keyword

In [ ]:
output_file_start = 'ukraine_russia_total_'

### Merging Tweets: Humanitarian Crisis

In [ ]:
df_humanitarian_crisis = merging_total('humanitarian_crisis')

In [ ]:
df_humanitarian_crisis

In [ ]:
# Dropping the column with the old indexes
df_humanitarian_crisis = df_humanitarian_crisis.drop(columns='Unnamed: 0')

In [ ]:
df_humanitarian_crisis

### Creating a count: Humanitarian Crisis

In [ ]:
df_humanitarian_crisis.dtypes

In [ ]:
# Converting created_at column to datetime data type
# This allows me to use datetime related functions, that make further processing easier
df_humanitarian_crisis['created_at'] = pd.to_datetime(df_humanitarian_crisis['created_at'])


In [ ]:
df_humanitarian_crisis

In [ ]:
# function to add one column for the date and hour they were created and one column that only contains the day
def adding_hour_and_day_column(df_input):
    # Creating an hour column for hourly count
    df_input['created_at_hour'] = pd.to_datetime(df_input['created_at'].apply(str).str[:-12])
    # Creating a day columnd for daily count
    df_input['created_at_day'] = pd.to_datetime(df_input['created_at']).dt.date
    
    return df_input
    

In [ ]:
df_humanitarian_crisis = adding_hour_and_day_column(df_humanitarian_crisis)

In [ ]:
df_humanitarian_crisis

In [ ]:
# Function to create dataframe with hourly count
def create_hour_count(df, keyword):
    df_hour_count = pd.DataFrame(df.groupby('created_at_hour').size())
    # This renames the count column after the keyword used
    df_hour_count.rename(columns={0: 'count_'+keyword}, inplace = True)
    
    return df_hour_count
    

In [ ]:
# Function to create dataframe with day count and rename the column with count (just as above)
def create_day_count(df, keyword):
    df_day_count = pd.DataFrame(df.groupby('created_at_day').size())
    df_day_count.rename(columns={0: 'count_'+keyword}, inplace = True)
    
    return df_day_count

In [ ]:
df_count_hour_hc = create_hour_count(df_humanitarian_crisis, 'humanitarian_crisis')

In [ ]:
df_count_hour_hc

In [ ]:
df_count_day_hc = create_day_count(df_humanitarian_crisis, 'humanitarian_crisis')

In [ ]:
df_count_day_hc

### Merging Tweets: Invasion

In [ ]:
df_invasion = merging_total('invasion')

In [ ]:
# Dropping the column with the old indexes
df_invasion = df_invasion.drop(columns='Unnamed: 0')

In [ ]:
df_invasion['created_at'] = pd.to_datetime(df_invasion['created_at'])

In [ ]:
df_invasion

In [ ]:
df_invasion.to_csv(output_file_start+'invasion.csv', index=False)

### Creating a count: Invasion

In [ ]:
df_invasion = adding_hour_and_day_column(df_invasion)

In [ ]:
df_invasion

In [ ]:
df_count_hour_invasion = create_hour_count(df_invasion, 'invasion')

In [ ]:
df_count_hour_invasion

I can see here, that there are some leftover tweets from 2022-02-23 23:00:00. They have to be dropped, as they don't belong to the defined time period.

In [ ]:
df_count_hour_invasion = df_count_hour_invasion.drop('2022-02-23 23:00:00')

In [ ]:
df_count_hour_invasion

Now it only contains the data it should.

In [ ]:
df_count_day_invasion = create_day_count(df_invasion, 'invasion')

In [ ]:
df_count_day_invasion

In [ ]:
# Here I have to drop 2022-02-23 for the same reasons. Here I just drop the first row by leaving out
df_count_day_invasion = df_count_day_invasion[1:]

In [ ]:
df_count_day_invasion

## Merging Tweets: Sanction

In [ ]:
df_sanction = merging_total('sanction')

In [ ]:
# Dropping the column with the old indexes
df_sanction = df_sanction.drop(columns='Unnamed: 0')

In [ ]:
df_sanction['created_at'] = pd.to_datetime(df_sanction['created_at'])

In [ ]:
df_sanction

In [ ]:
df_sanction.to_csv(output_file_start+'sanction.csv', index=False)

### Creating a count: Sanction

In [ ]:
df_sanction = adding_hour_and_day_column(df_sanction)

In [ ]:
df_sanction

In [ ]:
df_count_hour_sanction = create_hour_count(df_sanction, 'sanction')

In [ ]:
df_count_hour_sanction

In [ ]:
# I'm having the same problem with some excess data from 23 February, which needs to be dropped
df_count_hour_sanction = df_count_hour_sanction.drop('2022-02-23 23:00:00')

In [ ]:
# Checking that there is only the data left I need
df_count_hour_sanction

In [ ]:
df_count_day_sanction = create_day_count(df_sanction, 'sanction')

In [ ]:
df_count_day_sanction

In [ ]:
df_count_day_sanction = df_count_day_sanction.drop(df_count_day_sanction.index[[0]])

In [ ]:
df_count_day_sanction

## Merging Tweets: War

In [ ]:
df_war = merging_total('war')

In [ ]:
# Dropping the column with the old indexes
df_war = df_war.drop(columns='Unnamed: 0')

In [ ]:
df_war['created_at'] = pd.to_datetime(df_war['created_at'])

In [ ]:
df_war

In [ ]:
df_war.to_csv(output_file_start+'war.csv', index=False)

### Creating a count: War

In [ ]:
df_war = adding_hour_and_day_column(df_war)

In [ ]:
df_war

In [ ]:
df_count_hour_war = create_hour_count(df_war, 'war')

In [ ]:
df_count_hour_war

In [ ]:
df_count_hour_war = df_count_hour_war.drop('2022-02-23 23:00:00')

In [ ]:
df_count_hour_war

In [ ]:
df_count_day_war = create_day_count(df_war, 'war')

In [ ]:
df_count_day_war

In [ ]:
df_count_day_war = df_count_day_war.drop(df_count_day_war.index[[0]])

In [ ]:
df_count_day_war

## Merging hourly and daily counts

These are te hourly and daily dataframes I created:  

df_count_hour_hc  
df_count_hour_invasion   
df_count_hour_sanction   
df_count_hour_war   

     
     
df_count_day_hc  
df_count_day_invasion  
df_count_day_sanction  
df_count_day_war

In [ ]:
# Merging the hourly dataframes
df_hour_total = pd.concat([df_count_hour_hc, df_count_hour_invasion, df_count_hour_sanction, df_count_hour_war], axis=1)

In [ ]:
# Checking if it is complete
df_hour_total

In [ ]:
# saving it as a csv
df_hour_total.to_csv(output_file_start+'count_hour.csv', index=True)

In [ ]:
# Merging the daily dataframes
df_day_total = pd.concat([df_count_day_hc, df_count_day_invasion, df_count_day_sanction, df_count_day_war], axis=1)

In [ ]:
# Checking if it is complete
df_day_total

In [ ]:
# saving it as a csv
df_day_total.to_csv(output_file_start+'count_day.csv', index=True)